In [1]:
import sqlite3
import pandas as pd
from datetime import date

today = date.today().isoformat()

conn = sqlite3.connect("mlb_data.db")

# --- 1. Load today's schedule ---
df_games = pd.read_sql_query("""
    SELECT game_pk, official_date, game_datetime_utc, 
           home_team_id, home_team_name, away_team_id, away_team_name
    FROM schedule_silver
    WHERE official_date = ?
""", conn, params=[today])

print("✅ Schedule loaded:", len(df_games), "games")


✅ Schedule loaded: 15 games


In [2]:
# --- 2. Load standings into dict keyed like monolith expects ---
df_standings = pd.read_sql_query("""
    SELECT team_id, team_name, wins, losses, pct, games_back, wc_games_back, division_name
    FROM standings_silver
""", conn)

standings_dict = {}
for div_name, group in df_standings.groupby("division_name"):
    standings_dict[div_name] = {
        "teams": [
            {
                "team_id": row.team_id,
                "name": row.team_name,
                "w": row.wins,
                "l": row.losses,
                "gb": row.games_back,
                "wc_gb": row.wc_games_back,
            }
            for row in group.itertuples()
        ]
    }

conn.close()

print("✅ Standings dict ready with", len(standings_dict), "divisions")
print("Divisions:", list(standings_dict.keys())[:3], "…")


✅ Standings dict ready with 6 divisions
Divisions: ['American League Central', 'American League East', 'American League West'] …


In [3]:
# --- Helpers adapted to use standings_dict ---

def get_team_record(team_id, standings_data):
    """Return dict with name, winPct, gamesBack for a team_id from standings_dict."""
    for division in standings_data.values():
        for t in division['teams']:
            if t['team_id'] == team_id:
                w, l = t['w'], t['l']
                winPct = w / (w + l) if (w + l) > 0 else 0
                gb = t['gb']
                gb_val = 0.0 if gb in ("-", "+0", "0", None) else float(str(gb).replace("+", ""))
                return {
                    'name': t['name'],
                    'winPct': winPct,
                    'gamesBack': gb_val
                }
    return None

def _wc_gb_value(team_id, standings_data):
    """Return numeric Wild Card distance for a team (0.0 if tied/leading)."""
    for division in standings_data.values():
        for t in division['teams']:
            if t['team_id'] == team_id:
                raw = t.get('wc_gb', "0")
                if raw in ("-", "+0", "+0.0", "0", "0.0", None):
                    return 0.0
                try:
                    return abs(float(str(raw).replace("+", "")))
                except Exception:
                    return 999.0
    return 999.0

def _sliding_playoff_points(distance):
    """Convert distance (games back) into playoff points."""
    if distance <= 0:
        return 30
    elif distance >= 6:
        return 0
    elif distance <= 3:
        return 30 - (distance / 3) * 15
    else:
        return 15 - ((distance - 3) / 3) * 15

def _playoff_points_single(team_summary, team_id, standings_data):
    div_gb = team_summary['gamesBack']
    wc_gb = _wc_gb_value(team_id, standings_data)
    div_pts = _sliding_playoff_points(div_gb)
    wc_pts  = _sliding_playoff_points(wc_gb)
    return max(div_pts, wc_pts)

def _division_or_wc_head_to_head_bonus(home_id, away_id, standings_data):
    """Bonus if same division or both in WC hunt."""
    home = get_team_record(home_id, standings_data)
    away = get_team_record(away_id, standings_data)
    if not home or not away:
        return 0
    # Same division?
    for div_data in standings_data.values():
        team_ids = [t['team_id'] for t in div_data['teams']]
        if home_id in team_ids and away_id in team_ids:
            return 10
    # Wild Card proximity (within 3 games)
    home_wc = _wc_gb_value(home_id, standings_data)
    away_wc = _wc_gb_value(away_id, standings_data)
    if home_wc <= 3 and away_wc <= 3:
        return 10
    return 0


In [4]:
import sqlite3
from datetime import datetime
import pytz

def build_watchability_gold(db_path="mlb_data.db"):
    conn = sqlite3.connect(db_path)
    cur = conn.cursor()

    for _, g in df_games.iterrows():
        home_id = g.home_team_id
        away_id = g.away_team_id
        game_pk = g.game_pk
        game_date = g.official_date

        # Convert UTC → ET
        raw_iso = str(g.game_datetime_utc)
        utc_dt = datetime.fromisoformat(raw_iso.replace("Z", "+00:00"))
        local_dt = utc_dt.astimezone(pytz.timezone("US/Eastern"))
        game_time = local_dt.strftime("%I:%M %p").lstrip("0") + " ET"

        home_team = get_team_record(home_id, standings_dict)
        away_team = get_team_record(away_id, standings_dict)

        # --- Playoff points ---
        playoff_pts_home = _playoff_points_single(home_team, home_id, standings_dict) if home_team else 0
        playoff_pts_away = _playoff_points_single(away_team, away_id, standings_dict) if away_team else 0
        playoff_pts = max(playoff_pts_home, playoff_pts_away)

        if playoff_pts_home > 0 and playoff_pts_away > 0:
            if playoff_pts_home >= 20 and playoff_pts_away >= 20:
                playoff_pts += 15
            elif playoff_pts_home >= 10 and playoff_pts_away >= 10:
                playoff_pts += 10
            else:
                playoff_pts += 5

        playoff_pts += _division_or_wc_head_to_head_bonus(home_id, away_id, standings_dict)

        # --- Quality points ---
        quality_pts = 0
        if playoff_pts > 0 and home_team and away_team:
            if home_team['winPct'] >= 0.55 and away_team['winPct'] >= 0.55:
                quality_pts = 20
            elif home_team['winPct'] >= 0.55 or away_team['winPct'] >= 0.55:
                quality_pts = 10

        score = round(playoff_pts + quality_pts)

        # Insert into Gold
        cur.execute("""
            INSERT OR REPLACE INTO watchability_gold 
            (game_pk, game_date, game_time, home_team, away_team,
             playoff_pts, quality_pts, score, last_updated)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            game_pk, game_date, game_time,
            g.home_team_name, g.away_team_name,
            playoff_pts, quality_pts, score, datetime.now().isoformat()
        ))

    conn.commit()
    conn.close()
    print("✅ watchability_gold populated.")

# Run it
build_watchability_gold()


✅ watchability_gold populated.


In [5]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("mlb_data.db")

df = pd.read_sql_query("""
    SELECT game_date, game_time, home_team, away_team,
           playoff_pts, quality_pts, score
    FROM watchability_gold
    WHERE game_date = date('now')
    ORDER BY score DESC
""", conn)

conn.close()

print("✅ Today's watchability scores")
df


✅ Today's watchability scores


,game_date,game_time,home_team,away_team,playoff_pts,quality_pts,score
0,2025-09-09,7:07 PM ET,Toronto Blue Jays,Houston Astros,55.0,10.0,65.0
1,2025-09-09,8:05 PM ET,Texas Rangers,Milwaukee Brewers,55.0,10.0,65.0
2,2025-09-09,7:05 PM ET,New York Yankees,Detroit Tigers,45.0,20.0,65.0
3,2025-09-09,6:45 PM ET,Philadelphia Phillies,New York Mets,55.0,10.0,65.0
4,2025-09-09,10:10 PM ET,Los Angeles Dodgers,Colorado Rockies,40.0,10.0,50.0
5,2025-09-09,6:40 PM ET,Cleveland Guardians,Kansas City Royals,37.5,0.0,38.0
6,2025-09-09,9:40 PM ET,Seattle Mariners,St. Louis Cardinals,35.0,0.0,35.0
7,2025-09-09,9:40 PM ET,San Diego Padres,Cincinnati Reds,35.0,0.0,35.0
8,2025-09-09,9:45 PM ET,San Francisco Giants,Arizona Diamondbacks,30.0,0.0,30.0
9,2025-09-09,10:05 PM ET,Athletics,Boston Red Sox,15.0,10.0,25.0
